In [3]:
import os
from dotenv import load_dotenv

In [4]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [5]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [8]:
#%cd ..
%pwd

'd:\\GenAIProj\\MyProjs\\Interview-Question-Creator'

In [9]:
from langchain.document_loaders import PyPDFLoader # type: ignore

file_path="data/Java_Cheat_Sheet.pdf"
loader = PyPDFLoader(file_path)
data = loader.load()

In [10]:
len(data)

45

In [12]:
quest_gen = ""
for page in data:
    quest_gen += page.page_content  

In [13]:
from langchain.text_splitter import TokenTextSplitter
""" Keep the chunk size greater than 1. Hence, Chunk size should be smaller."""
que_gen_splitter = TokenTextSplitter(model_name= "gpt-3.5-turbo", chunk_size=5000, chunk_overlap=200)
chunk_ques_gen = que_gen_splitter.split_text(quest_gen)

In [14]:
"""Covert the string to Document for format to be passed to LLM.
Embedding Model doesn't support String."""

#len(chunk_ques_gen) -- list
#type(chunk_ques_gen) -- Str

from langchain.docstore.document import Document
document_ques_gen =  [Document(page_content = t) for t in chunk_ques_gen]

In [15]:
document_ques_gen

[Document(page_content='1\n \nCORE JAVA CHEATSHEET\n \n \n \nObject Oriented Programming Language:\n\u200b\n based on the concepts of “objects”.\n \nOpen Source: \n\u200b\nReadily available for development.\n \n \nPlatform-neutral: \n\u200b\nJava code is independent of any particular hardware or software. This is\n \nbecause Java code is compiled by the compiler and converted into byte code. Byte code is\n \nplatform-independent and can run on multiple systems. The only requirement is Java needs a\n \nruntime environment i.e, JRE, which is a set of tools used for developing Java applications.\n \n \nMemory Management: \n\u200b\nGarbage collected language, i.e. deallocation of memory.\n \nException Handling: \n\u200b\nCatches a series of errors or abnormality, thus eliminating any risk of\n \ncrashing the system.\n \nTHE JAVA BUZZWORDS\n \nJava was modeled in its final form keeping in consideration with the primary objective of having\n \nthe following features:\n \n \n \nSimple, Small 

In [16]:
ans_gen_splitter = TokenTextSplitter(model_name= "gpt-3.5-turbo", chunk_size=1000, chunk_overlap=100)

document_ans_gen = ans_gen_splitter.split_documents(document_ques_gen)

In [17]:
len(document_ans_gen)

14

In [18]:
""" Initialize the LLM. Lower temperature - More authentic and less creative. Higher temperature - More creative"""

"""The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. 
An updated version of the class exists in the langchain-openai package and should be used instead"""
#from langchain.chat_models import ChatOpenAI - Deprecated Now
#from langchain_openai import ChatOpenAI
from langchain.chat_models import ChatOpenAI

llm_ques_gen_ppline = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3)

d:\GenAIProj\MyProjs\Interview-Question-Creator\.venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [19]:
prompt_template = """
You are an expert at creating interview questions based on the study material and coding material given to you.
Your goal is to prepare a coder or a programmer for their exams and coding tests.
You do this by asking the questions about the text below:

--------
{text}
--------

Create a set of questions that will help the coders and programmers for their tests.
Make sure not to lose any important information.

QUESTIONS:
"""

In [20]:
from langchain.prompts import PromptTemplate
PROMPT_QUESTIONS = PromptTemplate(template=prompt_template, input_variables=['text'])

In [22]:
refine_template = ("""
You are an expert at creating interview questions based on the study material and coding material given to you.
Your goal is to prepare a coder or a programmer for their exams and coding tests.
We have received some questions meeting the expectations upto a certain extent: {existing_answers}.
We have the option to refine the existing questions or add new ones.
(only if necessary) with some more context below

--------
{text}
--------

Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.

QUESTIONS:
""")

In [23]:
REFINE_PROMPT_QUESTIONS = PromptTemplate(input_variables=["existing_answers", "text"], template=refine_template)

In [100]:
"""from langchain.chains.summarize import load_summarize_chain

ques_gen_chain = load_summarize_chain( llm=llm_ques_gen_ppline,
                                        chain_type="refine", 
                                        verbose=True,
                                        question_prompt=PROMPT_QUESTIONS,
                                        refine_prompt=REFINE_PROMPT_QUESTIONS)"""

In [25]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [26]:
# Create the prompt templates
PROMPT_QUESTIONS = PromptTemplate(template=prompt_template, input_variables=['text'])
REFINE_PROMPT_QUESTIONS = PromptTemplate(input_variables=["existing_answers", "text"], template=refine_template)


In [29]:
#ques = ques_gen_chain.run(document_ques_gen)

# Create the LLMChain for generating questions
ques_gen_chain = LLMChain(llm=llm_ques_gen_ppline, prompt=PROMPT_QUESTIONS)

In [30]:
# Run the chain with the document_ques_gen as input
ques = ques_gen_chain.run(document_ques_gen)

d:\GenAIProj\MyProjs\Interview-Question-Creator\.venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [32]:
print(ques)

1. What are the key features of Java as mentioned in the document?
2. Explain the concept of typecasting in Java with examples.
3. List and explain the different types of operators in Java as mentioned in the document.
4. Describe the process of executing Java code using an IDE and through the terminal as per the document.
5. What are the different types of variables in Java and how are they defined?
6. Explain the concept of inheritance in Java and provide examples of different types of inheritance.
7. How does polymorphism work in Java? Provide examples of method overriding and method overloading.
8. What is an abstract class in Java and how is it different from an interface?
9. Describe the concept of exception handling in Java and provide examples of common Java exceptions.
10. How does memory management work in Java? Explain the process of allocating and deallocating memory in Java.


In [37]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [38]:
embeddings = OpenAIEmbeddings()

In [39]:
vector_store = FAISS.from_documents(document_ans_gen, embeddings)

In [40]:
llm_ans_gen_ppline = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3)

In [41]:
ques_list = ques.split("\n")
ques_list

['1. What are the key features of Java as mentioned in the document?',
 '2. Explain the concept of typecasting in Java with examples.',
 '3. List and explain the different types of operators in Java as mentioned in the document.',
 '4. Describe the process of executing Java code using an IDE and through the terminal as per the document.',
 '5. What are the different types of variables in Java and how are they defined?',
 '6. Explain the concept of inheritance in Java and provide examples of different types of inheritance.',
 '7. How does polymorphism work in Java? Provide examples of method overriding and method overloading.',
 '8. What is an abstract class in Java and how is it different from an interface?',
 '9. Describe the concept of exception handling in Java and provide examples of common Java exceptions.',
 '10. How does memory management work in Java? Explain the process of allocating and deallocating memory in Java.']

In [42]:
from langchain.chains import RetrievalQA

In [43]:
ans_gen_chain = RetrievalQA.from_chain_type(llm=llm_ans_gen_ppline,
                                                chain_type="stuff",
                                                retriever=vector_store.as_retriever())

In [ ]:
for question in ques_list:
    print("Question: ", question)
    answer = ans_gen_chain.run(question)
    print("Answer: ", answer)
    print("------------------------------------------------\n")   
    
    #Saving Answers to the file 
    with open("answers.txt", "a") as answers_file:
        answers_file.write("Question: " + question + "\n")
        answers_file.write("Answers: " + answer + "\n")
        answers_file.write("------------------------------------------------\n")